<a href="https://colab.research.google.com/github/ShinAsakawa/ShinAsakawa.github.io/blob/master/2022notebooks/2022_0126onomatopea_bert_mlm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- filename: 2022_0126onomatopea_bert_mlm.ipynb
- memo: 2022年01月22日現在，
- author: 浅川伸一 asakawa@ieee.org
- lincense: MIT

* transformers は M1 Mac では動作しない。
具体的には，最適化関数 Adamw を呼び出すと halt する。
そこで，このコードでは，transformers 版の AdamW ではなく，PyTorch 版の AdamW を呼び出すように変更している。
ただし，ホスト名の判断は決め打ちしているので，各自変更しなければならない。

# 0 このコードの概要，ねらい

- huggingface が提供する `transformers` から BERT を呼び出す。
`transformers` に登録されているモデルのうち，東北大学乾研提供の 日本語化 BERT モデルを微調整 fine-tuning する。
モデル名としては `cl-tohoku/bert-base-japanese` である。

- この日本語化された BERT モデル (BERT-MLM) を `BertForMaskedLM` (マスク化言語モデルに特化した BERT) として呼び出し，オノマトペ予測課題としみなして訓練を行うことである。

## 本コードの具体的な手順

1. 必要なライブラリを輸入 import 
2. 小野編 「オノマトペ辞典4500」の読み込み
3. 訓練済 日本語 BERT モデルの読み込み
4. 日本語 BERT モデルで提供されているトークナイザに，小野編「オノマトペ辞典」を登録
5. 訓練テキストデータ (original.csv) の読み込み
6. 小野版オノマトペ辞典の，各オノマトペ記述文に出てくるオノマトペを [MASK] で置換する
7. PyTorch の流儀に従って Dataset, DataLoader を定義する
8. データセットを，訓練，検証，テストデータセットに 3 分割する
9. 最適化関数を定義する
10. 訓練と評価の定義
10. 結果の損失関数の減衰曲線を描画

---


# 1  必要なライブラリを輸入 import 

In [ ]:
import os
import sys
import numpy as np
import unicodedata
from termcolor import colored

# 本ファイルを Google Colaboratory 上で実行する場合に，必要となるライブラリをインストールする
import platform
isColab = platform.system() == 'Linux'
if isColab:
    !pip install transformers > /dev/null 2>&1 

    # MeCab, fugashi, ipadic のインストール
    !apt install aptitude swig > /dev/null 2>&1
    !aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y > /dev/null 2>&1
    !pip install mecab-python3 > /dev/null 2>&1
    !git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git > /dev/null 2>&1
    !echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n -a > /dev/null 2>&1
    
    import subprocess
    cmd='echo `mecab-config --dicdir`\"/mecab-ipadic-neologd\"'
    path_neologd = (subprocess.Popen(cmd, stdout=subprocess.PIPE,
                                     shell=True).communicate()[0]).decode('utf-8')

    !pip install 'fugashi[unidic]' > /dev/null 2>&1
    !python -m unidic download > /dev/null 2>&1
    !pip install ipadic > /dev/null 2>&1
    !pip install jaconv > /dev/null 2>&1
    !pip install japanize_matplotlib > /dev/null 2>&1    

In [ ]:
# PyTorch の seed の設定関連 再現性確保のため
# https://qiita.com/takubb/items/7d45ae701390912c7629
# https://qiita.com/si1242/items/d2f9195c08826d87d6ad
import numpy as np
import random
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

# リソースの選択（CPU/GPU）
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# 乱数シード固定（再現性の担保）
def fix_seed(seed):
    # random
    random.seed(seed)
    # numpy
    np.random.seed(seed)
    
    # pytorch\n",
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.random.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed = 42
fix_seed(seed)
# データローダーのサブプロセスの乱数のseedが固定
def worker_init_fn(worker_id):
    np.random.seed(np.random.get_state()[1][0] + worker_id)
    print(worker_init_fn(1))
    
# # データローダーの作成
# train_loader = torch.utils.data.DataLoader(train_dataset,
#                                            batch_size=16,  # バッチサイズ
#                                            shuffle=True,  # データシャッフル
#                                            num_workers=2,  # 高速化
#                                            pin_memory=True,  # 高速化
#                                            worker_init_fn=worker_init_fn
#                                            )

# 2 小野編 「オノマトペ辞典4500」の読み込み

In [ ]:
# 2021/Jan 近藤先生からいただいたオノマトペ辞典のデータの読み込み

#'日本語オノマトペ辞典4500より.xls' は著作権の問題があり，公にできません。
# そのため Google Colab での解法，ローカルファイルよりアップロードしてください
if isColab:
    from google.colab import files
    uploaded = files.upload()  # ここで `日本語オノマトペ辞典4500より.xls` を指定してアップロードする
    data_dir = '.'
else:
    data_dir = '/Users/asakawa/study/2021ccap/notebooks'

import pandas as pd
import jaconv

onomatopea_excel = '2021-0325日本語オノマトペ辞典4500より.xls'
onmtp2761 = pd.read_excel(os.path.join(data_dir, onomatopea_excel), sheet_name='2761語')

#すべてカタカナ表記にしてデータとして利用する場合
#`日本語オノマトペ辞典4500` はすべてひらがな表記だが，一般にオノマトペはカタカナ表記されることが多いはず
#onomatopea = list(sorted(set([jaconv.hira2kata(o) for o in onmtp2761['オノマトペ']])))

# Mac と Windows の表記の相違を吸収
onomatopea = list(sorted(set([jaconv.normalize(o) for o in onmtp2761['オノマトペ']])))
print(f'データファイル名: {os.path.join(data_dir, onomatopea_excel)}\n',
      f'オノマトペ単語総数: len(onomatopea):{len(onomatopea)}')

# 3 訓練済 日本語 BERT モデルの読み込みと，小野編「オノマトペ辞典」のトークナイザへの登録

In [ ]:
# transformers, huggingface 版の BERT 実装の読み込み
import torch
from transformers import BertConfig
from transformers import BertForPreTraining
from transformers import BertJapaneseTokenizer
from transformers import BertForMaskedLM

model_ja_name = 'cl-tohoku/bert-base-japanese'  # 東北大学乾研による 日本語 BERT 実装
model = BertForMaskedLM.from_pretrained(model_ja_name) # マスク化言語モデルを指定
config = BertConfig.from_pretrained(model_ja_name)

# GPU が利用可能であれば利用する
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

tknz1 = BertJapaneseTokenizer.from_pretrained(model_ja_name)
# BPE (or sentencepiece) による下位単語分割あり

# 4 日本語 BERT モデルで提供されているトークナイザに，小野編「オノマトペ辞典」を登録


In [ ]:
# トークナイザ の修正，実際には onomatopea 単語リストを引数に指定して `add_tokens()` を呼び出すだけ
# ただし，語彙数 tknz.vocab は変更されない。追加された語彙，本コードの場合はオノマトペは，
# `tknz1.added_tokens_encoder` と `tknz1.added_tokens_decoder` に反映されているためである
num_added = tknz1.add_tokens(onomatopea)
print(f'追加されたトークン数:{num_added}/オノマトペ数:{len(onomatopea)}') 
model.resize_token_embeddings(len(tknz1))

print(f' len(tknz1):{len(tknz1)}\n', 
      f'len(tknz1.vocab):{len(tknz1.vocab)}\n',  # 一見すると，この数字からオノマトペが追加されていないように見える。
      f'tknz1.vocab_size:{tknz1.vocab_size}')    # 駄菓子菓子，下で見るように，正しく動作しているように見受けられる

# print('# 確認用')
# for w in onomatopea[-5:]:
#     idx = tknz1.convert_tokens_to_ids(w)
#     w_ = tknz1.convert_ids_to_tokens(idx)
#     print(f'単語:{w}(id:{idx}) -> token:{w_}')    

# 5 訓練テキストデータ (original.csv) の読み込み

In [ ]:
# 近藤先生 (2021年12月22日） から送っていただいた，オノマトペ文章データ 'original.csv' を読み込む
import jaconv

if isColab:
    uploaded = files.upload()  # original.csv をアップロード
    data_dir = '.'
else:
    data_dir = '/Users/asakawa/study/2021kondo_project'

original = []
n = 0
with open(os.path.join(data_dir,'original.csv'), 'r', encoding='utf8') as f:
    s = f.read()
    for s_ in s.split('\n'):
        if n == 0:
            n += 1
            continue
        idx, sent = s_.split(',')
        
        # Mac と Windows との unicode 符号化の差分を吸収する
        # jaconv.normalize は内部で unicodedata.normalize('NFKC') を呼び出しているので
        # 差異 between Mac and Windows を吸収できる
        sent = ''.join(jaconv.normalize(x) for x in sent)
        original.append(sent)
        #original[int(idx)] = sent

print(f'{len(original)} has been read')

# 6 小野版オノマトペ辞典の，各オノマトペ記述文に出てくるオノマトペを [MASK] で置換する

In [ ]:
max_token_len = np.array([len(tknz1(s).input_ids) for s in original]).max()
max_token_len += 2  # 保険のため 2 くらい加えておく
print(f'max_token_len:{max_token_len}')

# トークナイザにかけて出力を得る。`max_length` のデフォルトは 512 だが，今回は長文である必要がないと考えられる。
# ここでは `max_token_len = 23` にしている。512 でも動作するが，学習に要する時間が増える
text = tuple(original)  # 全文をタプルに変換
inputs = tknz1(text, 
               return_tensors='pt', 
               max_length=max_token_len, 
               truncation=True, 
               padding='max_length')

#`labels` キーを追加する。実際には inputs_ids なのでラベルではなくトークンID の系列
inputs['labels'] = inputs.input_ids.detach().clone()

#トークン ID を走査して，オノマトペ単語であれば，[MASK] トークンに置き換える。
l_ = []
for l in inputs['labels']:
    l_.append([tknz1.mask_token_id if w in onomatopea else tknz1.convert_tokens_to_ids(w) for w in tknz1.convert_ids_to_tokens(l)])

inputs['input_ids'] = torch.LongTensor(l_)
#print(inputs['input_ids'].shape)

# 7 PyTorch の流儀に従って Dataset, DataLoader を定義する


In [ ]:
#データセットのためのクラスを定義
class onmtpDataset(torch.utils.data.Dataset):
    def __init__(self, encoder):
        self.encoder = encoder
        
    def __getitem__(self, idx):
        return {key:torch.tensor(val[idx]) for key, val in self.encoder.items()}
    
    def __len__(self):
        return len(self.encoder.input_ids)
    
dataset = onmtpDataset(inputs)

#データローダを準備
loader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True)

# GPU/CPU 使用を設定し，モデルの訓練モードを起動 #Setup GPU/CPU usage and activate the training mode of our model.
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device) # モデルを選択したデバイスに移動 # and move our model over to the selected device
model.train()  # 訓練モードに設定 #activate training mode

# 8 訓練データセットを，訓練，検証，テストデータセットの 3 つに分割する
<!-- # Split train dataset into train, validation and test sets -->

In [ ]:
#データセットを 7:1.5:1.5 に分割して 訓練データセット，検証データセット，テストデータセットに分割
train_size = int(dataset.__len__() * 0.70)
valid_size = int(dataset.__len__() * 0.15)
test_size = dataset.__len__() - train_size - valid_size

train_dataset, \
valid_dataset, \
test_dataset = torch.utils.data.random_split(dataset, 
                                             lengths=[train_size, test_size, valid_size], 
                                             generator=torch.Generator().manual_seed(seed))

# 9 最適化関数を定義する

In [ ]:
import socket 
#実行しているホスト名によって，M1 Mac であれば，Pytorch 版の AdamW を輸入し，そうでなければ Huggingface transformers 版の AdamW を輸入する  
if not 'Sinope' in socket.gethostname():
    # 以下だと M1 mac では halt する。
    #最適化関数を初期化 (AdamW は重み付き崩壊で，過学習の可能性を減らす) 
    #Initialize our optimizer (Adam with weighted decay - reduces chance of overfitting).
    from transformers import AdamW
    #最適化関数を初期化 # initialize optimizer
    optim = AdamW(model.parameters(), lr=5e-5)
else:
    # なので，transformers.AdamW ではなく，PyTorch の標準関数である Adam で代用する
    from torch.optim import AdamW

optim = AdamW(model.parameters(), lr=5e-5)

# 10 訓練と評価の定義

In [ ]:
import typing
import transformers

n_batch_size = 128
traindataset_loader = torch.utils.data.DataLoader(train_dataset, 
                                                  batch_size=n_batch_size, 
                                                  shuffle=True,
                                                  pin_memory=True,
                                                  worker_init_fn=worker_init_fn,
                                                 )
testdataset_loader  = torch.utils.data.DataLoader(test_dataset,  
                                                  batch_size=n_batch_size, 
                                                  shuffle=False,
                                                  pin_memory=True,
                                                  worker_init_fn=worker_init_fn,
                                                 )
validdataset_loader = torch.utils.data.DataLoader(valid_dataset, 
                                                  batch_size=n_batch_size, 
                                                  shuffle=False,
                                                  pin_memory=True,
                                                  worker_init_fn=worker_init_fn,
                                                 )

def forward(data:transformers.tokenization_utils_base.BatchEncoding, 
        model:transformers.models.bert.modeling_bert.BertForMaskedLM=model) -> transformers.modeling_outputs.MaskedLMOutput:
    _input_ids = data['input_ids'].clone().detach().to(device)  # ミニバッチサイズだけデータを取得
    _attention_mask = data['attention_mask'].clone().detach().to(device)
    _labels = data['labels'].clone().detach().to(device)
    _out = model(_input_ids,
                 attention_mask=_attention_mask,
                 labels=_labels)
    return _out


def eval(data:transformers.tokenization_utils_base.BatchEncoding, 
         model:transformers.models.bert.modeling_bert.BertForMaskedLM=model) -> transformers.modeling_outputs.MaskedLMOutput:
    model.eval()
    _input_ids = data['input_ids'].clone().detach().to(device)  # ミニバッチサイズだけデータを取得
    _attention_mask = data['attention_mask'].clone().detach().to(device)
    _labels = data['labels'].clone().detach().to(device)
    _out = model(_input_ids,
                 attention_mask=_attention_mask,
                 labels=_labels)
    return _out



# 11 訓練の実施

In [ ]:
%%time
from tqdm import tqdm

epochs = 40
#train_losses, valid_losses = [], []
for epoch in range(epochs):
    model.eval()
    valid_loss = 0.0
    valid_loop = tqdm(validdataset_loader)
    for data in valid_loop:
        _out = eval(data)
        _loss = _out.loss
        #valid_loss += _loss.item()
        valid_loss = _loss.item() # * data['input_ids'].size(0)
        valid_loop.set_description(f'\t検証エポック {epoch}') # 進行状況の表示
        valid_loop.set_postfix(loss=_loss.item())
    valid_losses.append(valid_loss)

    train_loss = 0.0
    model.train()
    train_loop = tqdm(traindataset_loader, leave=True)
    for data in train_loop:
        optim.zero_grad()   # 勾配情報の 0 クリア
        _out = forward(data)
        _loss = _out.loss   # 損失値を取得
        _loss.backward()    # 取得した損失値に基づいて BERT のパラメータを逆伝播
        optim.step()        # BERT パラメータの更新 すなわち学習
        train_loop.set_description(f'訓練エポック {epoch}') # 進行状況の表示
        train_loop.set_postfix(loss=_loss.item())
        train_loss += _loss.item()
    train_losses.append(train_loss)


# 12 損失の減衰曲線を描画

In [ ]:
import matplotlib.pyplot as plt
import japanize_matplotlib

plt.plot(train_losses[1:], color='red', label='訓練')
plt.plot(valid_losses[1:], color='green', label='検証')
plt.xlabel('訓練時間')
plt.ylabel('損失値')
plt.legend()
plt.title('オノマトペ微調整における学習の推移 (損失値) の減少')
plt.savefig('2022_0126onomatopea_mlm_train.pdf')
plt.show()

In [ ]:
#上記グラフのダウンロード
files.download('2022_0126onomatopea_mlm_train.pdf')

# 13 全オノマトペデータに対して予測を行う

In [ ]:
def eval_an_output(N, original=original, tknz1=tknz1, inputs=inputs, print_flag=True):
    """
    引数として 数字を 1 つ入力すると (N)，`original.csv` の N 行目のデータを読み込んで，
    その文のオノマトペを [MASK] に置き換えて，マスク化言語モデルで [MASK] を予測する。
    結果を表示する場合には 引数 `print_flag=True` として呼び出す
    """
    if N >= len(original) or (not isinstance(N, int)):
        return

    _out = model(inputs.input_ids[N].unsqueeze(0).to(device), attention_mask=inputs.attention_mask[N].unsqueeze(0).to(device), labels=inputs.labels[N].to(device))
    _x = _out.logits.detach()
    __x = _x.squeeze(0).detach().clone()
    _pred_idx  = torch.argmax(__x, dim=1, keepdim=True)
    _pred_s    = "/".join(tknz1.convert_ids_to_tokens(_pred_idx)).replace('/[PAD]','')
    
    _orig      = original[N] # 原文
    _inp_idx   = tknz1.convert_ids_to_tokens(inputs.input_ids[N]) # 入力トークンID
    _inp_s     = "/".join(_inp_idx).replace('/[PAD]','')          # 入力文
    _teach_idx = tknz1.convert_ids_to_tokens(inputs.labels[N])    # 教師信号トークンID
    _teach_s   = "/".join(_teach_idx).replace('/[PAD]','')        # 教師信号文
    
    _mask_pos = np.where(inputs.input_ids[N].detach().numpy() == tknz1.mask_token_id)
    _teach_tokens = inputs.labels[N][_mask_pos].detach().squeeze().numpy()
    _pred_tokens  = _pred_idx[_mask_pos].detach().squeeze().cpu().numpy()

    _n_hit = np.array([_teach_tokens == _pred_tokens]).sum()       # 正解したか否か
    if print_flag:
        color = 'grey' if _n_hit > 0 else 'red'
        print(f'{N:5,d}   原文:{_orig}')
        print(f'\t入力:{_inp_s}')
        print(f'\t正解:{_teach_s}')
        print(colored(f'\t出力:{_pred_s}',color))
        print(f'\tmask 位置:{_mask_pos}')
        print(f'\t正解トークン:{_teach_tokens}', f'予測トークン:{_pred_tokens}', 
              f'{np.array([_teach_tokens == _pred_tokens]).sum() > 0}')
        print(f'\t_out.loss:{_out.loss:.3f}')
    
    return _out.loss, _n_hit



In [ ]:
model.to(device)
model.eval()
total_hit = 0
for i in range(len(original)):
    _, hit = eval_an_output(i, print_flag=False) # print_flag = True にすると推論結果を表示します. 逆に False にすれば正解率だけ計算します
    total_hit += hit

print(f'正解数:{total_hit}/{len(original)}= {total_hit/len(original) * 100:.3f} %')    

# 14 任意の文章を入力して，オノマトペ語の予測を行う

In [ ]:
import typing

def eval_a_line(inp:str or list, 
                model:transformers.models.bert.modeling_bert.BertForMaskedLM=model, 
                tknz1:transformers.models.bert_japanese.tokenization_bert_japanese.BertJapaneseTokenizer=tknz1,
                onomatopea_vocab=onomatopea,
                max_token_len:int=max_token_len,
                print_flag:bool = True):
    """任意の文章を入れて，オノマトペを [MASK] に入れ替えて評価する"""
    if isinstance(inp, str):
        _inp = tknz1(inp,
                     return_tensors='pt', 
                     max_length=max_token_len, 
                     truncation=True, 
                     padding='max_length')
        
        #`labels` キーを追加する。実際には inputs_ids なのでラベルではなくトークン ID の系列
        _inp['labels'] = _inp.input_ids.detach().clone().squeeze()

        #トークン ID を走査して，オノマトペ単語であれば，[MASK] トークンに置き換える。
        _inp['input_ids'] = torch.LongTensor([tknz1.mask_token_id if w in onomatopea_vocab else tknz1.convert_tokens_to_ids(w) for w in tknz1.convert_ids_to_tokens(_inp['input_ids'].squeeze())]).unsqueeze(0)

        _str =  "".join(tknz1.convert_ids_to_tokens(_inp.input_ids.squeeze()))
        _orig = _str.replace('[CLS]','').replace('[PAD]','').replace('[SEP]','')
        
        _str =  "".join(tknz1.convert_ids_to_tokens(_inp.input_ids.squeeze()))
        _mask = _str.replace('[CLS]','').replace('[PAD]','').replace('[SEP]','')
        _out = model(_inp.input_ids.to(device), attention_mask=_inp.attention_mask.to(device), labels=_inp.labels.to(device))
        _logit = _out.logits.clone().detach().squeeze()
        _pred_idx  = torch.argmax(_logit, dim=1, keepdim=True)
        _pred_s    = "".join(tknz1.convert_ids_to_tokens(_pred_idx.squeeze())).replace('[PAD]','').replace('[CLS]','').replace('[SEP','')


        _positions =  _inp['input_ids'].clone().detach().squeeze().cpu().numpy()
        _mask_pos = np.where(_positions == tknz1.mask_token_id)
        _teach_tokens = _inp['labels'][_mask_pos].clone().detach().squeeze().cpu().numpy()
        _pred_tokens  = _pred_idx[_mask_pos].clone().detach().squeeze().cpu().numpy()
        _is_hit = np.array([_teach_tokens == _pred_tokens]).sum() > 0       # 正解したか否か

        if print_flag:
            print(f'ソース:{inp}')
            print(f'入力文:{_orig}')
            #print(f'マスク:{_mask}')
            print(f'予測文:{_pred_s}')
            (color, attrs) = ('red',['bold','reverse']) if not _is_hit else ('grey', ['bold'])
            print(f'({_teach_tokens}=={_pred_tokens})=', 
                  colored(f'{_is_hit}', color, attrs=attrs), 
                  colored(f'loss:{_out.loss:.3f}', color, attrs=attrs))
        return _is_hit, _out.loss
    elif isinstance(inp,list):
        return [eval_a_line(l) for l in inp]
    

In [ ]:
def dataset2list(_dataset:torch.utils.data.dataset.Subset=valid_dataset)-> list:
    """subdataset を list に変換する"""
    ret = []
    for d in _dataset:
        _s = tknz1.convert_ids_to_tokens(d['labels'].clone().detach())
        s = "".join(w for w in _s)
        ret.append(s.replace('[PAD]','').replace('[CLS]','').replace('[SEP]',''))
    return ret

_train = dataset2list(train_dataset)
_valid = dataset2list(valid_dataset)
_test  = dataset2list(test_dataset) 

# 15 訓練，検証，テスト，各データセットについての精度を計算

In [ ]:
total_hits = 0
for l in _train:
    (hit, loss) = eval_a_line(l, print_flag=False)
    total_hits += hit
print(f'total_hits:{total_hits}/len(train_dataset):{len(_train)}={total_hits/len(_train):.2f}%')

total_hits = 0
for l in _test:
    (hit, loss) = eval_a_line(l, print_flag=False)
    total_hits += hit
print(f'total_hits:{total_hits}/len(test_dataset):{len(_test)}={total_hits/len(_test):.2f}%')

total_hits = 0
for l in _valid:
    (hit, loss) = eval_a_line(l, print_flag=False)
    total_hits += hit
print(f'total_hits:{total_hits}/len(valid_dataset):{len(_valid)}={total_hits/len(_valid):.2f}%')

# 16 適当な文章を入力して結果を観察してみる

In [ ]:
#任意の文章を入力して，オノマトペ語の予測を行う

s = '今日の朝，彼女に壁どんした'
eval_a_line(s)

s = '私ははっ（と）驚いた。'
eval_a_line(s)

# X あと，上位 ｎ 個の予測を作らねばならんなー